In [1]:
import pandas as pd 
import numpy as np 

In [2]:
data = pd.read_csv('EURUSD15.csv', sep=',', names=['Time', 'Open', 'Max', 'Min', 'Close', 'Value'])

In [4]:
delta_max_min = pd.DataFrame((data['Max'].values - data['Min'].values), columns=['deltaMaxMin'])

In [5]:
data['DeltaMaxMin'] = data['Max'].values - data['Min'].values

In [6]:
del data['Max']
del data['Min']

In [7]:
mean_value = data['Value'].mean()
print mean_value
def make_mean(value):
    if (value >= mean_value):
        return 1
    else:
        return 0

94.2343324672


In [8]:
data['Value'] = data.apply(lambda row: make_mean(row['Value']), axis=1)

In [9]:
def make_time_for_usd_eur(time):
    if (not(1<=int(time[0:2])<=9)):
        return 1
    else:
        return 0


In [10]:
data['Time'] = data.apply(lambda row: make_time_for_usd_eur(row['Time']), axis=1)

In [11]:
count = 0;
y = []
Open = 0
for i in data['Open'].values:
    if (count%5 == 0):
        if (not(Open)):
            Open = i
        else:
            if (data['Close'].values[count] > Open):
                y.append(1)
                Open = i
            else:
                y.append(0)
                Open = i
    count+=1;


In [12]:
y_ = pd.DataFrame(y)

In [13]:
Y_data = y_


In [14]:
des = np.array(data.values,np.float32).reshape(len(data.values),5)

In [15]:
data_upd = pd.DataFrame(des)

In [16]:
data_upd = data_upd.drop(64655)

In [17]:
train = np.array(data_upd.values,np.float32).reshape(12931,25)

In [18]:
X_data = pd.DataFrame(train)

In [19]:
import tensorflow as tf
from sklearn import metrics, cross_validation

In [20]:
def to_vector(y, clas = 2):
    num_labels = len(y)
    index_off = np.arange(num_labels)*2
    labels_one_hot = np.zeros((num_labels, clas))
    labels_one_hot.flat[index_off + y] = 1
    return labels_one_hot

In [21]:
y_one_hot = to_vector(y, 2)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_data, y_one_hot, test_size=0.3, random_state=60)
y_one_hot

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [22]:
# Parameters
learning_rate = 0.001
batch_size = 125
display_step = 1

# Network Parameters
n_hidden_1 = 100 # 1st layer number of features
n_hidden_2 = 50 # 2nd layer number of features
n_input = 25 # data
n_classes = 2 #total classes (0-1 digits)

In [23]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [24]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.rnn(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.rnn(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [25]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
n_samples = list(X_train.shape)
cost = tf.reduce_sum((y - pred)**2/n_samples[0])
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

TypeError: rnn() takes at least 2 arguments (1 given)

In [ ]:
sess = tf.Session()

In [ ]:
init = tf.initialize_all_variables()
sess.run(init)

In [ ]:
for i in range(100):
    average_cost = 0
    number_of_bathes = int(len(X_train) / batch_size)
    for start, end in zip(range(0, len(X_train), batch_size), range(batch_size, len(X_train), batch_size)):
        sess.run(optimizer, feed_dict={x: X_train[start:end], y: y_train[start:end]})
        average_cost += sess.run(cost, feed_dict = {x: X_train[start:end], y: y_train[start:end]})
    print("Epoch:", '%04d' % (i+1), 'cost', "{:.9f}".format(average_cost))
print("Finished optimization!")
    

In [ ]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_test, 1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print('Accuracy: {0}'.format(sess.run(accuracy, feed_dict={x:X_test, y: y_test})))
